In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [2]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [3]:
# Model
def get_model():
    inputs = tf.keras.Input(shape=(54), name="dcdmm")
    x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128)(x)
    predictions = tf.keras.layers.Dense(7)(x)
    model = tf.keras.Model(inputs=inputs, outputs=predictions)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [4]:
model = get_model()
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 54)]              0         
                                                                 
 dense (Dense)               (None, 256)               14080     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable p

In [5]:
predictions = model.predict(X_test[:10],  # 预测10条样本
                            batch_size=32, verbose=0)
print('predictions shape:', predictions)

predictions shape: [[-1.2989109e+00  3.9603524e-03  8.5654885e-01  2.3984597e+00
   9.8658681e-01  7.5030875e-01 -4.7630601e+00]
 [ 5.3366909e+00  5.1405292e+00 -4.0203733e+00 -4.0420728e+00
   1.9598624e+00 -3.2058506e+00  3.9972854e+00]
 [-2.6332355e+00 -8.4017879e-01  1.8643417e+00  3.0384107e+00
   8.3566475e-01  1.7784278e+00 -7.2832060e+00]
 [-1.9307544e+00  3.2612193e-01  1.9112844e+00  1.2835355e+00
   2.2739542e+00  1.1376319e+00 -5.6657419e+00]
 [ 6.1730804e+00  5.5002098e+00 -3.6826839e+00 -4.6259551e+00
   3.5832069e+00 -3.6610520e+00  1.6343750e+00]
 [-2.6030662e+00 -3.2727695e-01  2.4359541e+00  1.7915792e+00
  -3.7647375e-01  2.0981703e+00 -6.4837513e+00]
 [-1.4121795e+00  7.0498705e-02  8.1566679e-01  2.3780968e+00
   1.5490258e+00  6.4440554e-01 -5.1706743e+00]
 [-2.4707813e+00 -2.5165939e-01  2.1654160e+00  1.6221936e+00
  -1.8705116e-01  2.2058997e+00 -6.2224092e+00]
 [-1.8845232e+00 -9.4623454e-03  1.8378738e+00  2.2638257e+00
   9.1935778e-01  9.3580639e-01 -5.9075

### 方法一:save_weights

In [6]:
# Saves all layer weights.
model.save_weights('save_weights.h5')

In [7]:
model_k_h5 = get_model()  # 重新创建网络
model_k_h5.load_weights('save_weights.h5')
model_k_h5.predict(X_test[:10], batch_size=32, verbose=0)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dcdmm (InputLayer)          [(None, 54)]              0         
                                                                 
 dense_4 (Dense)             (None, 256)               14080     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable

array([[-1.2989109e+00,  3.9603524e-03,  8.5654885e-01,  2.3984597e+00,
         9.8658681e-01,  7.5030875e-01, -4.7630601e+00],
       [ 5.3366909e+00,  5.1405292e+00, -4.0203733e+00, -4.0420728e+00,
         1.9598624e+00, -3.2058506e+00,  3.9972854e+00],
       [-2.6332355e+00, -8.4017879e-01,  1.8643417e+00,  3.0384107e+00,
         8.3566475e-01,  1.7784278e+00, -7.2832060e+00],
       [-1.9307544e+00,  3.2612193e-01,  1.9112844e+00,  1.2835355e+00,
         2.2739542e+00,  1.1376319e+00, -5.6657419e+00],
       [ 6.1730804e+00,  5.5002098e+00, -3.6826839e+00, -4.6259551e+00,
         3.5832069e+00, -3.6610520e+00,  1.6343750e+00],
       [-2.6030662e+00, -3.2727695e-01,  2.4359541e+00,  1.7915792e+00,
        -3.7647375e-01,  2.0981703e+00, -6.4837513e+00],
       [-1.4121795e+00,  7.0498705e-02,  8.1566679e-01,  2.3780968e+00,
         1.5490258e+00,  6.4440554e-01, -5.1706743e+00],
       [-2.4707813e+00, -2.5165939e-01,  2.1654160e+00,  1.6221936e+00,
        -1.8705116e-01,  

### 方法二:save
* 保存和加载整个模型

In [8]:
# Saves the model to Tensorflow SavedModel or a single HDF5 file.
model.save("save.h5")  # HDF5 file

# ★★★★★推荐保存为SavedModel格式
# model.save('keras_model_tf_version', save_format='tf')

In [9]:
new_model = tf.keras.models.load_model('save.h5')  # 不需要重新创建网络
new_model.predict(X_test[:10], batch_size=32, verbose=0)

array([[-1.2989109e+00,  3.9603524e-03,  8.5654885e-01,  2.3984597e+00,
         9.8658681e-01,  7.5030875e-01, -4.7630601e+00],
       [ 5.3366909e+00,  5.1405292e+00, -4.0203733e+00, -4.0420728e+00,
         1.9598624e+00, -3.2058506e+00,  3.9972854e+00],
       [-2.6332355e+00, -8.4017879e-01,  1.8643417e+00,  3.0384107e+00,
         8.3566475e-01,  1.7784278e+00, -7.2832060e+00],
       [-1.9307544e+00,  3.2612193e-01,  1.9112844e+00,  1.2835355e+00,
         2.2739542e+00,  1.1376319e+00, -5.6657419e+00],
       [ 6.1730804e+00,  5.5002098e+00, -3.6826839e+00, -4.6259551e+00,
         3.5832069e+00, -3.6610520e+00,  1.6343750e+00],
       [-2.6030662e+00, -3.2727695e-01,  2.4359541e+00,  1.7915792e+00,
        -3.7647375e-01,  2.0981703e+00, -6.4837513e+00],
       [-1.4121795e+00,  7.0498705e-02,  8.1566679e-01,  2.3780968e+00,
         1.5490258e+00,  6.4440554e-01, -5.1706743e+00],
       [-2.4707813e+00, -2.5165939e-01,  2.1654160e+00,  1.6221936e+00,
        -1.8705116e-01,  

### 方法三:tf.savaed_model.save
* 保存和加载整个模型

In [10]:
# 一般用于模型部署
tf.saved_model.save(model, 'tf_saved_model_save')

INFO:tensorflow:Assets written to: tf_saved_model_save\assets


In [11]:
!saved_model_cli show --dir tf_saved_model_save --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 54)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(shape=(None

In [12]:
'''
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 54)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_27'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
'''
restored_saved_model = tf.saved_model.load('tf_saved_model_save')
f = restored_saved_model.signatures["serving_default"]
f(dcdmm=tf.cast(X_test[:10], tf.float32))

{'dense_3': <tf.Tensor: shape=(10, 7), dtype=float32, numpy=
 array([[-1.2989109e+00,  3.9603524e-03,  8.5654885e-01,  2.3984597e+00,
          9.8658681e-01,  7.5030875e-01, -4.7630601e+00],
        [ 5.3366909e+00,  5.1405292e+00, -4.0203733e+00, -4.0420728e+00,
          1.9598624e+00, -3.2058506e+00,  3.9972854e+00],
        [-2.6332355e+00, -8.4017879e-01,  1.8643417e+00,  3.0384107e+00,
          8.3566475e-01,  1.7784278e+00, -7.2832060e+00],
        [-1.9307544e+00,  3.2612193e-01,  1.9112844e+00,  1.2835355e+00,
          2.2739542e+00,  1.1376319e+00, -5.6657419e+00],
        [ 6.1730804e+00,  5.5002098e+00, -3.6826839e+00, -4.6259551e+00,
          3.5832069e+00, -3.6610520e+00,  1.6343750e+00],
        [-2.6030662e+00, -3.2727695e-01,  2.4359541e+00,  1.7915792e+00,
         -3.7647375e-01,  2.0981703e+00, -6.4837513e+00],
        [-1.4121795e+00,  7.0498705e-02,  8.1566679e-01,  2.3780968e+00,
          1.5490258e+00,  6.4440554e-01, -5.1706743e+00],
        [-2.4707813e+0